In [3]:
import pandas as pd
import numpy as np

In [58]:
action = pd.read_csv('./Action.csv')
clf = pd.read_csv('./data_csv/Modularity.csv')
idx2id = pd.read_csv('./data_csv/Index2Id.csv')

action['score'] = 0

print(action.head())
print(clf.head())
print(idx2id.head())

   Unnamed: 0    repo_id  actor_id    actor_login              type   action  \
0           0  159513310   9887585     snellingio        WatchEvent  started   
1           1  167922316   9887585     snellingio        WatchEvent  started   
2           2  170486632   9887585     snellingio        WatchEvent  started   
3           3  105722352   9887716       vpperego        WatchEvent  started   
4           4  158469400   9887743  ElliottMiller  PullRequestEvent   opened   

          pull_merged  score  
0  ('pull_merged', 0)      0  
1  ('pull_merged', 0)      0  
2  ('pull_merged', 0)      0  
3  ('pull_merged', 0)      0  
4  ('pull_merged', 0)      0  
   Id  Label  timeset  indegree  outdegree  degree  pageranks  \
0   0    NaN      NaN         0          7       7   0.000006   
1   1    NaN      NaN         2          0       2   0.000007   
2   2    NaN      NaN         1          0       1   0.000007   
3   3    NaN      NaN         1          0       1   0.000007   
4   4   

In [59]:
def score(x):
    res = 0
    if x.type == 'IssuesEvent':
        res = 1.
        if x.action == 'opened':
            res *= 2
    elif x.type == 'PullRequestEvent':
        res = 2.
        if x.pull_merged == 1:
            res *= 2.5
    elif x.type == 'PullRequestReviewCommentEvent':
        res = 4.
    elif x.type == 'WatchEvent':
        res = 1.
    else:
        res = 2.
    return res

action['score'] = action.apply(score, axis = 1)

In [60]:
action[['type', 'action', 'score']]

,type,action,score
0,WatchEvent,started,1.0
1,WatchEvent,started,1.0
2,WatchEvent,started,1.0
3,WatchEvent,started,1.0
4,PullRequestEvent,opened,2.0
...,...,...,...
99995,WatchEvent,started,1.0
99996,ForkEvent,NaN,2.0
99997,WatchEvent,started,1.0
99998,WatchEvent,started,1.0


In [61]:
clf1 = pd.merge(clf, idx2id, left_on = 'Id', right_on = 'index')[['user_id', 'modularity_class']]
clf1
clf1['user_id'] = clf1['user_id'].astype('int64')
action = pd.merge(clf1, action, how = 'right', left_on = 'user_id', right_on = 'actor_id')


In [69]:
action = action.fillna(method = 'ffill')
action
action.to_csv('./data_csv/Action.csv', index = 0)

In [1]:
from read_data import read_data
repo_name, repo_data = read_data()

print(len(repo_name), len(repo_data))
repo_name[0:10]

In [42]:
action = pd.read_csv('./data_csv/Action.csv')
vec = []
num_classes = len(action['modularity_class'].unique())
for repo_id in repo_name:
    vec_item = np.zeros((num_classes,))
    series = action.loc[action['repo_id'] == repo_id]
    group = series.groupby('modularity_class')
    idx = np.array(list(group.indices.keys())).astype('int')
    # print(np.array(list(group.indices.keys())).astype('int'))
    vec_item[idx] = np.sqrt(list(group.sum()['score']))
    # print(vec_item[vec_item > 0])
    vec.append(vec_item)
    
vec = np.array(vec)
print(vec.shape)

(24287, 1638)


In [1]:
from get_dataset import get_dataset

dset = get_dataset()

Now loading text data...
Now loading contribute vector data, this may take minutes.
Text data's length:  24287
Vector's shape:  (24287, 1638)
